# Music Recommendation System
## Part 2: Modeling & Recommendation Engine

This notebook implements **various embedding techniques and recommendation strategies** for our content-based music recommendation system. We compare TF-IDF, Word2Vec, FastText, and Sentence-BERT embeddings for lyric-based song similarity.

---

## Table of Contents

### Core Implementation (Completed)
1. [Import Libraries](#1-import-libraries)
2. [Load Dataset](#2-load-dataset)
3. [Baseline: TF-IDF](#3-baseline-tf-idf)
4. [Building FAISS Index](#4-building-faiss-index-baseline)
5. [Recommendation Function](#5-recommendation-function)
6. [Model Persistence](#6-model-persistence)

### Advanced Embeddings (Completed)
7. [Word2Vec & FastText](#7-improvement-1-word2vec--fasttext)
   - [7.1 Tokenize Lyrics for Word Embeddings](#71-tokenize-lyrics-for-word-embeddings)
   - [7.2 Train Custom Word2Vec Model](#72-train-custom-word2vec-model)
   - [7.3 Document Embedding Function](#73-document-embedding-function)
   - [7.4 Build Document Embeddings for All Songs](#74-build-document-embeddings-for-all-songs)
   - [7.5 Train Custom FastText Model](#75-train-custom-fasttext-model)
   - [7.6 Explore Semantic Relationships](#76-explore-semantic-relationships)
   - [7.7 Build FastText Document Embeddings](#77-build-fasttext-document-embeddings)
   - [7.8 Build FAISS Indexes for Semantic Search](#78-build-faiss-indexes-for-semantic-search)
   - [7.9 Semantic Recommendation Functions](#79-semantic-recommendation-functions)
   - [7.10 Test the Recommendation System](#710-test-the-recommendation-system)
   - [7.11 Save Semantic Models](#711-save-semantic-models)

8. [Sentence-BERT](#8-improvement-2-sentence-bert)
   - [8.1 Load Sentence-Transformers](#81-load-sentence-transformers)
   - [8.2 Generate SBERT Embeddings for All Songs](#82-generate-sbert-embeddings-for-all-songs)
   - [8.3 Build FAISS Index for SBERT Embeddings](#83-build-faiss-index-for-sbert-embeddings)
   - [8.4 SBERT Recommendation Function](#84-sbert-recommendation-function)
   - [8.5 Test the Recommendation System](#85-test-the-recommendation-system)
   - [8.6 Save SBERT Models and Embeddings](#86-save-sbert-models-and-embeddings)

### Future Enhancements (Optional)
9. [Audio Features (Spotify API)](#9-improvement-3-audio-features-spotify-api) - *TODO*
10. [Embedding Comparison & Evaluation](#10-embedding-comparison--evaluation) - *TODO*
11. [Hybrid Recommendation System](#11-hybrid-recommendation-system) - *TODO*
12. [Advanced FAISS Indexes](#12-advanced-faiss-indexes) - *TODO*

---

## Summary

**Implemented Models:**
- **TF-IDF**: Baseline bag-of-words approach (5000 dimensions)
- **Word2Vec**: Custom-trained on lyrics dataset (100 dimensions)
- **FastText**: Custom-trained with subword support (100 dimensions)
- **SBERT**: Sentence-BERT contextual embeddings (384 dimensions)

**All models are saved and ready for deployment!**



## 1. Import Libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import normalize
import faiss
import pickle
from scipy.sparse import save_npz
import gensim.downloader as api
from gensim.models import Word2Vec, FastText
import random
from tqdm import tqdm
from huggingface_hub import HfApi, login
from sentence_transformers import SentenceTransformer
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

## 2. Load Dataset

In [2]:
df = pd.read_csv('../data/processed/spotify_millsongdata_processed.csv')

## 3. Baseline: TF-IDF

> **Baseline Approach**: We start with TF-IDF (Term Frequency-Inverse Document Frequency) as our baseline embedding method. This is a classic approach for text similarity that works well but has limitations in capturing semantic meaning.

**TF-IDF (Term Frequency-Inverse Document Frequency)** is a numerical statistic that reflects how important a word is to a document in a collection.

- **Term Frequency (TF)**: How often a word appears in a document

$$
tf(t, d) = \frac{f_{t,d}}{\sum_{k} f_{k,d}}
$$

where $f_{t,d}$ denotes the number of occurrences of term $t$ in document $d$.

- **Inverse Document Frequency (IDF)**: How rare a word is across all documents

$$
idf(t, D) = \log \left( \frac{N}{1 + n_t} \right)
$$

where:
- $N$ is the total number of documents in the corpus,
- $n_t$ is the number of documents containing term $t$.


**TF-IDF Weight**

The final TF-IDF weight of term $t$ in document $d$ is computed as:

$$
w_{t,d} = tf(t, d) \times idf(t, D)
$$

Words that appear frequently in one song but rarely across all songs get higher weights, making them better for distinguishing between songs.

We limit to `max_features=5000` to keep the most important terms and reduce dimensionality.

In [3]:
tfidf_vectorizer = TfidfVectorizer(max_features=5000, dtype=np.float32)
tfidf_matrix = tfidf_vectorizer.fit_transform(df['cleaned_text'])

print(f"TF-IDF Matrix Shape: {tfidf_matrix.shape}")
print(f"Vocabulary Size: {len(tfidf_vectorizer.vocabulary_)}")

TF-IDF Matrix Shape: (57650, 5000)
Vocabulary Size: 5000


## 4. Building FAISS Index (Baseline)

**FAISS (Facebook AI Similarity Search)** is a library for efficient similarity search and clustering of dense vectors.

### **Why FAISS?**
- Traditional cosine similarity has O(n) complexity for each query
- FAISS uses optimized algorithms for faster nearest neighbor search
- Essential for scaling to large datasets (millions of songs)

---

### **Index Type: `IndexFlatIP`**
- **IP** = Inner Product (equivalent to cosine similarity for L2-normalized vectors)
- **Flat** = Exact search (no approximation)
- We normalize vectors with L2 norm so inner product equals cosine similarity

#### **$L_2$ Normalization**

$L_2$ normalization transforms a document vector  $\mathbf{x} = [x_1, x_2, \dots, x_n]$  into a unit vector $\hat{\mathbf{x}}$ with Euclidean norm equal to 1:

$$
\hat{\mathbf{x}} = \frac{\mathbf{x}}{\|\mathbf{x}\|_2}
= \frac{\mathbf{x}}{\sqrt{\sum_{i=1}^{n} x_i^2}}
$$

##### **Geometric Interpretation**

After $L_2$ normalization, all document vectors lie on the surface of a **unit hypersphere**. This normalization removes the influence of document length, ensuring that similarity comparisons depend only on the **direction** of the vectors (semantic content), rather than their magnitude.

---

#### **Similarity Search with FAISS and Inner Product**

In FAISS, `IndexFlatIP` performs similarity search based on the **Inner Product (IP)**.  
Given a query vector $\mathbf{q}$ and a set of document vectors $\mathbf{X}$ stored in the index, the similarity score $s_i$ is computed as:

$$
s_i = \mathbf{q} \cdot \mathbf{x}_i
= \sum_{j=1}^{dim} q_j \cdot x_{i,j}
$$

##### **Relationship Between Inner Product and Cosine Similarity**

Cosine Similarity between two vectors $\mathbf{q}$ and $\mathbf{x}$ is defined as:

$$
\text{cosine\_sim}(\mathbf{q}, \mathbf{x})
= \frac{\mathbf{q} \cdot \mathbf{x}}{\|\mathbf{q}\|_2 \, \|\mathbf{x}\|_2}
$$

Since both the query vector and document vectors are $L_2$-normalized  ($\|\mathbf{q}\|_2 = 1$ and $\|\mathbf{x}\|_2 = 1$), the equation simplifies to:

$$
\text{cosine\_sim}(\mathbf{q}, \mathbf{x})
= \mathbf{q} \cdot \mathbf{x}
= \text{Inner Product}
$$

Therefore, when using `IndexFlatIP` on $L_2$-normalized TF-IDF vectors, FAISS effectively computes **Cosine Similarity**, representing the degree of semantic similarity between the query text and the retrieved documents.

In [4]:
X = tfidf_matrix.toarray()
X = normalize(X, norm="l2")

dim = X.shape[1]
print(f"Vector Dimension: {dim}")

index = faiss.IndexFlatIP(dim)
index.add(X)

print(f"Total vectors in FAISS index: {index.ntotal}")
print("FAISS index built successfully!")

Vector Dimension: 5000
Total vectors in FAISS index: 57650
FAISS index built successfully!


## 5. Recommendation Function

The core recommendation logic:

1. **Find the song**: Look up the input song name in our dataset
2. **Get its vector**: Retrieve the TF-IDF vector for that song
3. **Search similar songs**: Use FAISS to find the k most similar songs
4. **Return results**: Return song details (excluding the input song itself)

In [5]:
def recommend_songs_tfidf(song_name, df=df, top_k=5):

    idx = df[df['song'].str.lower() == song_name.lower()].index
    if len(idx) == 0:
        return "Song not found in the dataset."
    idx = idx[0]

    query = X[idx].reshape(1, -1)

    scores, indices = index.search(query, top_k + 1)

    results = df[['song', 'artist']].iloc[indices[0][1:]].copy()
    results['similarity'] = scores[0][1:]

    return results

### Test the Recommendation System

Let's test our recommendation system by getting similar songs for the first song in our dataset.

In [6]:
idx = random.randint(0, len(df))
test_song = df['song'].iloc[idx]
test_artist = df['artist'].iloc[idx]
print(f"Getting recommendations for: '{test_song}' by {test_artist}")
print("-" * 50)

display(recommend_songs_tfidf(test_song))

Getting recommendations for: 'All Those Years Ago' by George Harrison
--------------------------------------------------


,song,artist,similarity
5860,Stone,Faces,0.603980
13920,Ultraviolence,New Order,0.472739
4824,Two Years Ago,Ellie Goulding,0.452838
42284,Remember Me,Lucky Dube,0.438898
51099,It Wasn't Very Long Ago,Roy Orbison,0.397654


## 6. Model Persistence

Save the trained models and processed data for later use:

- **df_cleaned.pkl**: Preprocessed DataFrame with cleaned lyrics
- **faiss_index.pkl**: FAISS index for fast similarity search
- **tfidf_matrix.pkl**: TF-IDF vectors for all songs

These files can be loaded later to make recommendations without retraining.

In [7]:
df.to_parquet("../data/processed/df_cleaned.parquet", index=False)
print("Saved: df_cleaned.parquet")

save_npz("../data/embeddings/embeddings_tfidf.npz", tfidf_matrix)
print("Saved: embeddings_tfidf.npz")

faiss.write_index(index, "../models/faiss_indexes/faiss_tfidf.index")
print("Saved: faiss_tfidf.index")

print("\nAll models saved successfully!")

Saved: df_cleaned.parquet
Saved: embeddings_tfidf.npz
Saved: faiss_tfidf.index

All models saved successfully!


---
# IMPROVEMENTS

The following sections implement advanced techniques to improve recommendation quality and performance.


## 7. Improvement 1: Word2Vec / FastText

> **Goal**: Capture semantic similarity between words. Unlike TF-IDF which treats words as independent tokens, word embeddings represent words in a continuous vector space where semantically similar words are close together.

### Why Word Embeddings?

**Limitation of TF-IDF vs Solution with Word2Vec/FastText:**
- "happy" and "joyful" are completely different features in TF-IDF, but similar vectors in Word2Vec
- TF-IDF has no semantic understanding, Word2Vec captures word meaning
- OOV words are ignored in TF-IDF, FastText handles via subwords

### Approaches

1. **Pre-trained Word2Vec** (Google News 300d)
   - 3M words, 300 dimensions
   - General semantic knowledge
   
2. **Custom FastText** (trained on lyrics)
   - Domain-specific vocabulary
   - Handles misspellings and slang


### 7.1 Tokenize Lyrics for Word Embeddings


In [8]:
df['tokens'] = df['cleaned_text'].apply(lambda x: x.split())

df[['song', 'text', 'tokens']]

,song,text,tokens
0,Ahe's My Kind Of Girl,"Look at her face, it's a wonderful face \r\nA...","[look, face, wonderful, face, means, something..."
1,"Andante, Andante","Take it easy with me, please \r\nTouch me gen...","[take, easy, please, touch, gently, like, summ..."
2,As Good As New,I'll never know why I had to go \r\nWhy I had...,"[ill, never, know, go, put, lousy, rotten, sho..."
3,Bang,Making somebody happy is a question of give an...,"[making, somebody, happy, question, give, take..."
4,Bang-A-Boomerang,Making somebody happy is a question of give an...,"[making, somebody, happy, question, give, take..."
...,...,...,...
57645,Good Old Days,Irie days come on play \r\nLet the angels fly...,"[irie, days, come, play, let, angels, fly, let..."
57646,Hand To Mouth,Power to the workers \r\nMore power \r\nPowe...,"[power, workers, power, power, workers, need, ..."
57647,Come With Me,all you need \r\nis something i'll believe \...,"[need, something, ill, believe, flashlights, h..."
57648,Desire,northern star \r\nam i frightened \r\nwhere ...,"[northern, star, frightened, go, rest, cant, s..."


### 7.2 Train Custom Word2Vec Model

We train a Word2Vec model directly on our lyrics dataset to capture domain-specific semantic relationships.

**Why Custom Training?**
- Domain-specific vocabulary (music lyrics, slang, artist names)
- Smaller model size (100 dimensions vs 300)
- Better understanding of music-specific terms
- No external dependencies or large downloads

**Training Parameters:**
- `vector_size=100`: Dimension of word vectors (balanced between quality and efficiency)
- `window=5`: Context window size (5 words before and after)
- `min_count=2`: Ignore words that appear less than 2 times
- `workers=4`: Parallel training with 4 threads
- `sg=1`: Use Skip-gram algorithm (better for small datasets)
- `epochs=10`: Number of training iterations

In [9]:
try:
    # w2v_model = api.load('word2vec-google-news-300')
    w2v_model = Word2Vec(
        sentences=df['tokens'].to_list(),
        vector_size=100,
        window=5,
        min_count=2,
        workers=4,
        sg=1,
        epochs=10
    )
    print(f"Vocabulary size: {len(w2v_model.key_to_index):,}")
    print(f"Vector dimension: {w2v_model.vector_size}")
except Exception as e:
    print(f"Error loading model: {e}")

Error loading model: 'Word2Vec' object has no attribute 'key_to_index'


### 7.3 Document Embedding Function

To get a single vector representation for an entire song (document), we compute the **weighted average** of word vectors:

$$\vec{d} = \frac{1}{|W|} \sum_{w \in W} \vec{w}$$

where $W$ is the set of words in the document and $\vec{w}$ is the word vector.

In [10]:
def get_document_embedding(tokens, model):
    """
    Compute document embedding by averaging word vectors.
    
    Args:
        tokens: List of words in the document
        model: Word2Vec/FastText model
        vector_size: Dimension of word vectors
    
    Returns:
        numpy array of shape (vector_size,) - document embedding
    """

    vector_size = model.vector_size
    word_vectors = []
    for word in tokens:
        if word in model:
            word_vectors.append(model[word])
     
    if len(word_vectors) == 0:
        return np.zeros(vector_size)
    
    return np.mean(word_vectors, axis=0)

In [11]:
sample_tokens = df['tokens'].iloc[0]
sample_embedding = get_document_embedding(sample_tokens, w2v_model.wv)
print(f"Sample document embedding shape: {sample_embedding.shape}")
print(f"First 10 values: {sample_embedding[:10]}")

Sample document embedding shape: (100,)
First 10 values: [-0.32375708  0.17338105  0.09558945  0.15887557  0.05994973 -0.07022716
  0.2102842   0.32940835 -0.27039382 -0.16260444]


### 7.4 Build Document Embeddings for All Songs

In [12]:
doc_embedding_w2v = []
for tokens in tqdm(df['tokens'], desc="Processing songs"):
    embedding = get_document_embedding(tokens, w2v_model.wv)
    doc_embedding_w2v.append(embedding)

X_w2v = np.array(doc_embedding_w2v, dtype=np.float32)

print(f"Document embeddings shape: {X_w2v.shape}")
print(f"Memory usage: {X_w2v.nbytes / 1024 / 1024:.2f} MB")

Processing songs: 100%|██████████| 57650/57650 [00:08<00:00, 6989.57it/s]

Document embeddings shape: (57650, 100)
Memory usage: 21.99 MB


### 7.5 Train Custom FastText Model


FastText extends Word2Vec by representing words as bags of character n-grams. This allows it to:
- Handle **out-of-vocabulary (OOV)** words by composing subword vectors
- Better handle **misspellings** and **slang** common in lyrics
- Capture **morphological** relationships (e.g., "love", "loving", "loved")

**Training Parameters:**
- `vector_size=100`: Dimension of word vectors (smaller for faster training)
- `window=5`: Context window size
- `min_count=2`: Ignore words that appear less than 2 times
- `epochs=10`: Number of training iterations
- `sg=1`: Use Skip-gram (better for small datasets)

In [13]:
fasttext_model = FastText(
    sentences=df['tokens'].to_list(),
    vector_size=100,
    window=5,
    min_count=2,
    workers=4,
    sg=1,
    epochs=10
)

print(f"FastText model trained successfully!")
print(f"Vocabulary size: {len(fasttext_model.wv.key_to_index):,}")
print(f"Vector dimension: {fasttext_model.wv.vector_size}")

FastText model trained successfully!
Vocabulary size: 52,953
Vector dimension: 100


### 7.6 Explore Semantic Relationships

Let's verify that our FastText model captured meaningful semantic relationships.

In [14]:
test_words = ['love', 'heart', 'dance', 'night', 'baby']

print("Words most similar to common lyrics terms:\n")
for word in test_words:
    try:
        similar = fasttext_model.wv.most_similar(word, topn=5)
        similar_words = [f"{w} ({s:.3f})" for w, s in similar]
        print(f"'{word}' → {', '.join(similar_words)}")
    except KeyError:
        print(f"'{word}' not in vocabulary")

Words most similar to common lyrics terms:

'love' → itlove (0.773), ohlove (0.767), lovewe (0.760), mylove (0.756), alove (0.755)
'heart' → heartfelt (0.786), heartmy (0.785), heartheart (0.767), hearts (0.736), hearttheres (0.724)
'dance' → dancego (0.824), dancefloor (0.759), decadance (0.758), danceevery (0.743), danceflo (0.736)
'night' → nightgo (0.821), nightynight (0.821), nightoh (0.803), nightgive (0.791), nightnight (0.786)
'baby' → bbaby (0.805), ohbaby (0.797), babyi (0.792), bbbbaby (0.789), ofbaby (0.784)


### 7.7 Build FastText Document Embeddings

Now let's create document embeddings using our trained FastText model.

In [15]:
print("Computing FastText document embeddings for all songs...")

doc_embeddings_ft = []
for tokens in tqdm(df['tokens'], desc="Processing songs"):
    embedding = get_document_embedding(tokens, fasttext_model.wv)
    doc_embeddings_ft.append(embedding)

X_ft = np.array(doc_embeddings_ft, dtype=np.float32)

print(f"\nFastText document embeddings shape: {X_ft.shape}")
print(f"Memory usage: {X_ft.nbytes / 1024 / 1024:.2f} MB")

Computing FastText document embeddings for all songs...


Processing songs: 100%|██████████| 57650/57650 [00:10<00:00, 5704.94it/s]


FastText document embeddings shape: (57650, 100)
Memory usage: 21.99 MB


### 7.8 Build FAISS Indexes for Semantic Search

Create FAISS indexes for both Word2Vec and FastText embeddings.

In [16]:
# Normalize embeddings for cosine similarity
X_w2v_norm = normalize(X_w2v, norm='l2')
X_ft_norm = normalize(X_ft, norm='l2')

# Build FAISS index for Word2Vec embeddings
print("Building FAISS index for Word2Vec embeddings...")
index_w2v = faiss.IndexFlatIP(X_w2v_norm.shape[1])
index_w2v.add(X_w2v_norm)
print(f"Word2Vec FAISS index: {index_w2v.ntotal} vectors, {X_w2v_norm.shape[1]} dimensions")

# Build FAISS index for FastText embeddings
print("\nBuilding FAISS index for FastText embeddings...")
index_ft = faiss.IndexFlatIP(X_ft_norm.shape[1])
index_ft.add(X_ft_norm)
print(f"FastText FAISS index: {index_ft.ntotal} vectors, {X_ft_norm.shape[1]} dimensions")

Building FAISS index for Word2Vec embeddings...
Word2Vec FAISS index: 57650 vectors, 100 dimensions

Building FAISS index for FastText embeddings...
FastText FAISS index: 57650 vectors, 100 dimensions


### 7.9 Semantic Recommendation Functions

Create recommendation functions using Word2Vec and FastText embeddings.

In [17]:
def recommend_songs_w2v(song_name, df=df, top_k=5):
    """
    Recommend songs using Word2Vec embeddings.
    """
    idx = df[df['song'].str.lower() == song_name.lower()].index
    if len(idx) == 0:
        return "Song not found in the dataset."
    idx = idx[0]
    
    query = X_w2v_norm[idx].reshape(1, -1)
    scores, indices = index_w2v.search(query, top_k + 1)
    
    result = df[['song', 'artist']].iloc[indices[0][1:]].copy()
    result['similarity'] = scores[0][1:]
    return result


def recommend_songs_fasttext(song_name, df=df, top_k=5):
    """
    Recommend songs using FastText embeddings.
    """
    idx = df[df['song'].str.lower() == song_name.lower()].index
    if len(idx) == 0:
        return "Song not found in the dataset."
    idx = idx[0]
    
    query = X_ft_norm[idx].reshape(1, -1)
    scores, indices = index_ft.search(query, top_k + 1)
    
    result = df[['song', 'artist']].iloc[indices[0][1:]].copy()
    result['similarity'] = scores[0][1:]
    return result


### 7.10 Test the Recommendation System

In [18]:
idx = random.randint(0, len(df))
test_song = df['song'].iloc[idx]
test_artist = df['song'].iloc[idx]

print(f"Recommendations for: '{test_song}' by {test_artist}")
print("=" * 70)

print("\nWord2Vec Recommendations:")
print("-" * 40)
display(recommend_songs_w2v(test_song))

print("\nFastText Recommendations:")
print("-" * 40)
display(recommend_songs_fasttext(test_song))

Recommendations for: 'Chaos At The Airport' by Chaos At The Airport

Word2Vec Recommendations:
----------------------------------------


,song,artist,similarity
14559,The Last Time,O.A.R.,0.964723
33405,Alien Afternoon,Genesis,0.964573
37362,From Silver Lake,Jackson Browne,0.963711
20095,It's Dangerous Business Walking Out Your Front...,Underoath,0.963548
48107,More Than This,Peter Gabriel,0.962642



FastText Recommendations:
----------------------------------------


,song,artist,similarity
33405,Alien Afternoon,Genesis,0.967511
14559,The Last Time,O.A.R.,0.963286
12472,The Great Escape,Marillion,0.963130
34448,Black Throated Wind,Grateful Dead,0.961648
45512,Waiting For The Sirens' Call,New Order,0.961460


### 7.11 Save Semantic Models

In [19]:
# Save FastText embeddings
np.savez_compressed("../data/embeddings/embeddings_fasttext.npz", X_ft)
print("Saved: embeddings_fasttext.npz")

# Save Word2Vec embeddings (from pre-trained model)
np.savez_compressed("../data/embeddings/embeddings_w2v.npz", X_w2v)
print("Saved: embeddings_w2v.npz")

# Save FAISS indexes
faiss.write_index(index_w2v, "../models/faiss_indexes/faiss_w2v.index")
print("Saved: faiss_w2v.index")

faiss.write_index(index_ft, "../models/faiss_indexes/faiss_fasttext.index")
print("Saved: faiss_fasttext.index")

print("\nAll semantic models saved successfully!")

Saved: embeddings_fasttext.npz
Saved: embeddings_w2v.npz
Saved: faiss_w2v.index
Saved: faiss_fasttext.index

All semantic models saved successfully!


---

## 8. Improvement 2: Sentence-BERT

> **Goal**: Use transformer-based embeddings that understand entire sentences/paragraphs, not just individual words.

### Why Sentence-BERT?

**Word2Vec Limitation vs SBERT Solution:**
- Word2Vec averages word vectors (loses word order), SBERT encodes full sentence context
- "I love you" = "You love I" in Word2Vec, but SBERT preserves word order meaning
- Word2Vec has fixed vocabulary, SBERT handles any text

### Model Options

1. **`all-MiniLM-L6-v2`** (recommended)
   - 384 dimensions
   - Fast inference
   - Good quality

2. **`all-mpnet-base-v2`** (higher quality)
   - 768 dimensions
   - Slower but more accurate


### 8.1 Load Sentence-Transformers


In [20]:
sbert_model = SentenceTransformer(r'all-MiniLM-L6-v2')

print(f"Max sequence length: {sbert_model.max_seq_length}")
print(f"Embedding dimension: {sbert_model.get_sentence_embedding_dimension()}")

Max sequence length: 256
Embedding dimension: 384


### 8.2 Generate SBERT Embeddings for All Songs


In [21]:
lyrics_texts = df['cleaned_text'].tolist()

X_sbert = sbert_model.encode(
    lyrics_texts,
    show_progress_bar=True,
    batch_size=64,
    convert_to_numpy=True
)

X_sbert = X_sbert.astype(np.float32)

print(f"\nSBERT embeddings shape: {X_sbert.shape}")
print(f"Memory usage: {X_sbert.nbytes / 1024 / 1024:.2f} MB")

Batches:   0%|          | 0/901 [00:00<?, ?it/s]


SBERT embeddings shape: (57650, 384)
Memory usage: 84.45 MB


### 8.3 Build FAISS Index for SBERT Embeddings


In [22]:
X_sbert_norm = normalize(X_sbert, norm='l2')

print("Building FAISS index for SBERT embeddings...")
index_sbert = faiss.IndexFlatIP(X_sbert_norm.shape[1])
index_sbert.add(X_sbert_norm)
print(f"SBERT FAISS index: {index_sbert.ntotal} vectors, {X_sbert_norm.shape[1]} dimensions")

Building FAISS index for SBERT embeddings...
SBERT FAISS index: 57650 vectors, 384 dimensions


### 8.4 SBERT Recommendation Function

In [23]:
def recommend_songs_sbert(song_name, df=df, top_k=5):
    idx = df[df['song'].str.lower() == song_name.lower()].index
    if len(idx) == 0:
        return "Song not found in the dataset"
    idx = idx[0]

    query = X_sbert_norm[idx].reshape(1, -1)

    scores, indices = index_sbert.search(query, top_k + 1)

    result = df[['song', 'artist']].iloc[indices[0][1:]].copy()
    result['similarity'] = scores[0][1:]

    return result

### 8.5 Test the Recommendation System

In [24]:
# Pick a random song for testing
idx = random.randint(0, len(df) - 1)
test_song = df['song'].iloc[idx]
test_artist = df['artist'].iloc[idx]

print(f"Recommendations for: '{test_song}' by {test_artist}")
print("=" * 80)

display(recommend_songs_sbert(test_song))

Recommendations for: 'Gangbang Rookie' by Snoop Dogg


,song,artist,similarity
18609,Gangsta Ride,Snoop Dogg,0.836990
51826,Get Bout It Rowdy,Snoop Dogg,0.835201
51825,Gangsta Walk,Snoop Dogg,0.832067
41631,Don't Die,Lil Wayne,0.831523
46310,Player's Anthem,Notorious B.I.G.,0.826387


### 8.6 Save SBERT Models and Embeddings

In [25]:
# Save SBERT embeddings
np.savez_compressed("../data/embeddings/embeddings_sbert.npz", X_sbert)
print("Saved: embeddings_sbert.npz")

# Save SBERT FAISS index
faiss.write_index(index_sbert, "../models/faiss_indexes/faiss_sbert.index")
print("Saved: faiss_sbert.index")

print("\nSBERT models saved successfully!")

Saved: embeddings_sbert.npz
Saved: faiss_sbert.index

SBERT models saved successfully!


---

## 9. Improvement 3: Audio Features (Spotify API)

> **Goal**: Incorporate audio characteristics (tempo, energy, danceability) to capture musical similarity beyond lyrics.

### Spotify Audio Features

- `danceability`: How suitable for dancing (0.0 - 1.0)
- `energy`: Intensity and activity (0.0 - 1.0)
- `tempo`: Beats per minute (~50 - 200)
- `valence`: Musical positiveness (0.0 - 1.0)
- `acousticness`: Acoustic vs electronic (0.0 - 1.0)
- `instrumentalness`: Vocal vs instrumental (0.0 - 1.0)

### Implementation Steps

1. Set up Spotify API credentials
2. Fetch audio features for each song
3. Normalize and combine with lyrics embeddings
4. Create hybrid similarity metric


### 9.1 Set Up Spotify API


In [26]:
# TODO: Install spotipy and set up credentials
# !pip install spotipy -q
# import spotipy
# from spotipy.oauth2 import SpotifyClientCredentials
# sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id="...", client_secret="..."))


### 9.2 Fetch Audio Features for Songs


In [27]:
# TODO: Search for tracks and get audio features
# def get_audio_features(artist, song):
#     results = sp.search(q=f"artist:{artist} track:{song}", type='track', limit=1)
#     if results['tracks']['items']:
#         track_id = results['tracks']['items'][0]['id']
#         return sp.audio_features(track_id)[0]
#     return None


### 9.3 Create Audio Feature Embeddings


In [28]:
# TODO: Normalize and combine audio features into embeddings
# audio_features = ['danceability', 'energy', 'tempo', 'valence', 'acousticness', 'instrumentalness']
# X_audio = df[audio_features].values
# X_audio = normalize(X_audio, norm='l2')


---

## 10. Embedding Comparison & Evaluation

> **Goal**: Compare TF-IDF, Word2Vec, and SBERT embeddings for lyric-based song similarity. Evaluate which method produces the most meaningful recommendations.

### Evaluation Metrics

- **Qualitative**: Manual inspection of recommendation quality
- **Diversity**: How diverse are the recommended songs?
- **Artist Coverage**: Do recommendations include different artists?
- **Semantic Coherence**: Do lyrics share similar themes?


### 10.1 Side-by-Side Comparison


In [29]:
# TODO: Compare recommendations from all methods for the same song
# def compare_recommendations(song_name, top_k=5):
#     print(f"Recommendations for: {song_name}")
#     print("\n📊 TF-IDF:")
#     display(recommend_songs_tfidf(song_name, top_k))
#     print("\n📊 Word2Vec:")
#     display(recommend_songs_w2v(song_name, top_k))
#     print("\n📊 Sentence-BERT:")
#     display(recommend_songs_sbert(song_name, top_k))


### 10.2 Embedding Visualization (t-SNE/UMAP)


In [30]:
# TODO: Visualize embeddings in 2D using t-SNE or UMAP
# from sklearn.manifold import TSNE
# X_embedded = TSNE(n_components=2).fit_transform(X_sbert[:1000])
# plt.scatter(X_embedded[:, 0], X_embedded[:, 1], alpha=0.5)
# plt.title("SBERT Embeddings (t-SNE)")


### 10.3 Summary Table


In [31]:
# TODO: Create summary comparison table
# comparison_df = pd.DataFrame({
#     'Method': ['TF-IDF', 'Word2Vec', 'FastText', 'SBERT'],
#     'Dimensions': [5000, 300, 100, 384],
#     'Semantic': ['❌', '✅', '✅', '✅✅'],
#     'Speed': ['Fast', 'Fast', 'Fast', 'Slow'],
#     'Quality': ['Good', 'Better', 'Better', 'Best']
# })


---

## 11. Hybrid Recommendation System

> **Goal**: Combine multiple similarity signals (lyrics, genre, artist, popularity) for better recommendations.

### Hybrid Score Formula

$$\text{FinalScore} = w_1 \cdot \text{LyricSim} + w_2 \cdot \text{GenreSim} + w_3 \cdot \text{Popularity}$$

**Default Weights:**
- w1 = 0.5 (Lyrics similarity - most important)
- w2 = 0.3 (Genre/Artist similarity)
- w3 = 0.2 (Popularity score)

### Components to Combine

- **Lyrics Similarity**: SBERT/TF-IDF cosine similarity (0.5)
- **Artist Similarity**: Same/similar artist bonus (0.1)
- **Genre Similarity**: Same genre bonus (0.2)
- **Popularity**: Normalized play count (0.1)
- **Release Year**: Recency bonus (0.1)


### 11.1 Lyrics Similarity Component


In [32]:
# TODO: Get lyrics similarity scores from SBERT index
# def get_lyrics_similarity(song_idx, candidate_indices):
#     query = X_sbert[song_idx].reshape(1, -1)
#     scores, _ = index_sbert.search(query, len(candidate_indices))
#     return scores[0]


### 11.2 Artist / Genre Similarity Component


In [33]:
# TODO: Compute artist/genre similarity
# def get_artist_similarity(song_idx, candidate_indices):
#     query_artist = df.iloc[song_idx]['artist']
#     similarities = []
#     for idx in candidate_indices:
#         if df.iloc[idx]['artist'] == query_artist:
#             similarities.append(1.0)  # Same artist
#         else:
#             similarities.append(0.0)
#     return np.array(similarities)


### 11.3 Popularity / Release Year Component


In [34]:
# TODO: Compute popularity scores (if available)
# def get_popularity_scores(candidate_indices):
#     if 'popularity' in df.columns:
#         scores = df.iloc[candidate_indices]['popularity'].values
#         return (scores - scores.min()) / (scores.max() - scores.min())
#     return np.zeros(len(candidate_indices))


### 11.4 Hybrid Recommendation Function


In [35]:
# TODO: Implement hybrid recommendation
# def recommend_songs_hybrid(song_name, top_k=5, weights={'lyrics': 0.5, 'genre': 0.3, 'popularity': 0.2}):
#     idx = df[df['song'].str.lower() == song_name.lower()].index[0]
#     
#     # Get initial candidates from lyrics similarity
#     _, candidates = index_sbert.search(X_sbert[idx].reshape(1, -1), 100)
#     candidates = candidates[0][1:]  # Exclude query song
#     
#     # Compute component scores
#     lyrics_scores = get_lyrics_similarity(idx, candidates)
#     artist_scores = get_artist_similarity(idx, candidates)
#     popularity_scores = get_popularity_scores(candidates)
#     
#     # Combine with weights
#     final_scores = (weights['lyrics'] * lyrics_scores + 
#                     weights['genre'] * artist_scores + 
#                     weights['popularity'] * popularity_scores)
#     
#     # Return top-k
#     top_indices = np.argsort(final_scores)[::-1][:top_k]
#     return df.iloc[candidates[top_indices]][['song', 'artist']]


### 11.5 Weight Tuning & Optimization


In [36]:
# TODO: Test different weight combinations
# weight_configs = [
#     {'lyrics': 0.5, 'genre': 0.3, 'popularity': 0.2},
#     {'lyrics': 0.7, 'genre': 0.2, 'popularity': 0.1},
#     {'lyrics': 0.4, 'genre': 0.4, 'popularity': 0.2},
# ]
# 
# for config in weight_configs:
#     print(f"Weights: {config}")
#     display(recommend_songs_hybrid(test_song, weights=config))


---

## 12. Advanced FAISS Indexes

> **Goal**: Compare different FAISS index types for speed vs accuracy trade-offs.

### Index Types Comparison

- **IndexFlatIP**: Exact search (baseline) - Slow, High Memory, 100% Recall
- **IndexIVFFlat**: Inverted file index - Fast, Medium Memory, ~95% Recall
- **IndexHNSWFlat**: Hierarchical NSW graph - Very Fast, Medium Memory, ~98% Recall
- **IndexIVFPQ**: Product quantization - Very Fast, Low Memory, ~90% Recall

### When to Use Each

- **IndexFlatIP**: Small datasets (<100K), need exact results
- **IndexIVFFlat**: Medium datasets, good balance
- **IndexHNSWFlat**: Large datasets, best quality/speed
- **IndexIVFPQ**: Very large datasets, memory constrained


### 12.1 Baseline: IndexFlatIP (Current)


In [37]:
# Baseline: Exact search
# index_flat = faiss.IndexFlatIP(dim)
# index_flat.add(X)
# print(f"IndexFlatIP: {index_flat.ntotal} vectors")


### 12.2 IndexIVFFlat (Inverted File Index)


In [38]:
# TODO: Build IndexIVFFlat
# nlist = 100  # Number of clusters
# quantizer = faiss.IndexFlatIP(dim)
# index_ivf = faiss.IndexIVFFlat(quantizer, dim, nlist, faiss.METRIC_INNER_PRODUCT)
# index_ivf.train(X)
# index_ivf.add(X)
# index_ivf.nprobe = 10  # Number of clusters to search
# print(f"IndexIVFFlat: {index_ivf.ntotal} vectors, {nlist} clusters")


### 12.3 IndexHNSWFlat (Hierarchical NSW)


In [39]:
# TODO: Build IndexHNSWFlat
# M = 32  # Number of connections per layer
# index_hnsw = faiss.IndexHNSWFlat(dim, M, faiss.METRIC_INNER_PRODUCT)
# index_hnsw.hnsw.efConstruction = 40  # Construction-time parameter
# index_hnsw.hnsw.efSearch = 16  # Search-time parameter
# index_hnsw.add(X)
# print(f"IndexHNSWFlat: {index_hnsw.ntotal} vectors, M={M}")


### 12.4 Speed Comparison Benchmark


In [40]:
# TODO: Benchmark search speed
# import time
# 
# def benchmark_index(index, name, X, n_queries=100, top_k=10):
#     queries = X[np.random.choice(len(X), n_queries)]
#     start = time.time()
#     for q in queries:
#         index.search(q.reshape(1, -1), top_k)
#     elapsed = time.time() - start
#     qps = n_queries / elapsed
#     print(f"{name}: {qps:.1f} queries/sec ({elapsed*1000/n_queries:.2f} ms/query)")
#     return qps
# 
# benchmark_index(index_flat, "IndexFlatIP", X)
# benchmark_index(index_ivf, "IndexIVFFlat", X)
# benchmark_index(index_hnsw, "IndexHNSWFlat", X)


### 12.5 Memory Usage Comparison


In [41]:
# TODO: Compare memory usage
# import sys
# 
# def get_index_size(index):
#     faiss.write_index(index, "/tmp/temp_index")
#     size_mb = os.path.getsize("/tmp/temp_index") / 1024 / 1024
#     return size_mb
# 
# print(f"IndexFlatIP: {get_index_size(index_flat):.2f} MB")
# print(f"IndexIVFFlat: {get_index_size(index_ivf):.2f} MB")
# print(f"IndexHNSWFlat: {get_index_size(index_hnsw):.2f} MB")


### 12.6 Recall@K Evaluation


In [42]:
# TODO: Compute Recall@K (how many of exact top-K are found by ANN)
# def compute_recall(index_exact, index_approx, X, n_queries=100, k=10):
#     queries = X[np.random.choice(len(X), n_queries)]
#     recalls = []
#     for q in queries:
#         q = q.reshape(1, -1)
#         _, exact_ids = index_exact.search(q, k)
#         _, approx_ids = index_approx.search(q, k)
#         recall = len(set(exact_ids[0]) & set(approx_ids[0])) / k
#         recalls.append(recall)
#     return np.mean(recalls)
# 
# print(f"IndexIVFFlat Recall@10: {compute_recall(index_flat, index_ivf, X):.2%}")
# print(f"IndexHNSWFlat Recall@10: {compute_recall(index_flat, index_hnsw, X):.2%}")


### 12.7 Performance Comparison Chart


In [43]:
# TODO: Create visualization comparing all indexes
# results = pd.DataFrame({
#     'Index': ['IndexFlatIP', 'IndexIVFFlat', 'IndexHNSWFlat'],
#     'Speed (QPS)': [100, 1000, 5000],
#     'Memory (MB)': [500, 550, 600],
#     'Recall@10': [1.0, 0.95, 0.98]
# })
# 
# fig, axes = plt.subplots(1, 3, figsize=(15, 4))
# results.plot.bar(x='Index', y='Speed (QPS)', ax=axes[0], color='steelblue')
# results.plot.bar(x='Index', y='Memory (MB)', ax=axes[1], color='coral')
# results.plot.bar(x='Index', y='Recall@10', ax=axes[2], color='seagreen')
# plt.tight_layout()
# plt.show()
